In [348]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from datetime import datetime
from keras.callbacks import EarlyStopping
pd.set_option('display.max_columns', 500)
from sklearn.feature_selection import SelectKBest, chi2, f_classif

In [349]:
def kelly(p, b, weight):
    stake = round((p * (b - 1) - (1 - p)) / (b - 1), 4) * weight
    return stake if stake > 0.0 else 'no value'

In [427]:
today = f"{datetime.now():%Y%m%d}"
print(today)
df = pd.read_csv(f'data_{today}.csv', index_col = [0]).fillna(0)
df = df[df['odds_checker_sum'] > 1.02]
# df = df.drop(['full_odds'], axis = 1)
print(df.shape)

20220911
(275, 51)


In [436]:
df = pd.read_csv('f_eng_df.csv', index_col = [0])#.dropna(axis = 1)
df = df[df['odds_sum'] > 1.02]
df = df.dropna(axis = 1)

In [437]:
df

,expected_goals_(xg)_diff,expected_goals_(xg)_sum,total_shots_diff,total_shots_sum,big_chances_diff,big_chances_sum,big_chances_missed_diff,big_chances_missed_sum,accurate_passes_diff,accurate_passes_sum,accurate_passes_percentage_diff,accurate_passes_percentage_sum,fouls_committed_diff,fouls_committed_sum,offsides_diff,offsides_sum,corners_diff,corners_sum,shots_off_target_diff,shots_off_target_sum,shots_on_target_diff,shots_on_target_sum,blocked_shots_diff,blocked_shots_sum,hit_woodwork_diff,hit_woodwork_sum,shots_inside_box_diff,shots_inside_box_sum,shots_outside_box_diff,shots_outside_box_sum,passes_diff,passes_sum,own_half_diff,own_half_sum,opposition_half_diff,opposition_half_sum,accurate_long_balls_diff,accurate_long_balls_sum,accurate_long_balls_percentage_diff,accurate_long_balls_percentage_sum,accurate_crosses_diff,accurate_crosses_sum,throws_diff,throws_sum,yellow_cards_diff,yellow_cards_sum,red_cards_diff,red_cards_sum,tackles_won_diff,tackles_won_sum,interceptions_diff,interceptions_sum,blocks_diff,blocks_sum,clearances_diff,clearances_sum,keeper_saves_diff,keeper_saves_sum,duels_won_diff,duels_won_sum,ground_duels_won_diff,ground_duels_won_sum,aerial_duels_won_diff,aerial_duels_won_sum,aerial_duels_won_percentage_diff,aerial_duels_won_percentage_sum,successful_dribbles_diff,successful_dribbles_sum,score_diff,score_sum,posession_diff,posession_sum,odds_sum,odds_predict,std_0,std_1,std_2,med_0,med_1,med_2,target
0,0.89,1.11,7.0,9.0,1.0,1.0,0.0,0.0,77.0,309.0,0.06,1.68,-3.0,13.0,1.0,1.0,1.0,5.0,4.0,4.0,0.0,2.0,3.0,3.0,0.0,0.0,7.0,9.0,0.0,0.0,77.0,365.0,43.0,183.0,34.0,126.0,8.0,24.0,0.13,0.97,3.0,5.0,2.0,14.0,-2.0,2.0,0.0,0.0,-3.0,13.0,2.0,8.0,-3.0,3.0,-1.0,19.0,0.0,0.0,-2.0,54.0,-1.0,43.0,-1.0,11.0,-0.10,1.00,4.0,10.0,0.0,2.0,20.0,100.0,1.091726,0,0.075452,0.045693,0.212633,2.300000,2.387500,4.200000,0
1,-1.21,1.47,-5.0,15.0,-1.0,1.0,0.0,0.0,-54.0,262.0,-0.09,1.59,9.0,19.0,-1.0,1.0,-1.0,7.0,-1.0,7.0,-2.0,4.0,-2.0,4.0,0.0,0.0,-4.0,8.0,-1.0,7.0,-49.0,327.0,5.0,129.0,-59.0,133.0,-7.0,27.0,-0.15,1.15,-1.0,3.0,-3.0,11.0,2.0,4.0,0.0,0.0,5.0,7.0,0.0,8.0,2.0,4.0,9.0,19.0,0.0,2.0,-3.0,57.0,-5.0,41.0,2.0,16.0,0.06,0.94,-2.0,8.0,-1.0,3.0,-16.0,100.0,1.087449,2,0.286912,1.108553,0.021276,4.750000,13.000000,1.250000,2
2,-0.74,1.08,-3.0,9.0,-1.0,1.0,0.0,0.0,-140.0,422.0,-0.11,1.61,-3.0,9.0,0.0,0.0,-1.0,3.0,0.0,4.0,-1.0,3.0,-2.0,2.0,0.0,0.0,-1.0,5.0,-2.0,4.0,-141.0,515.0,-31.0,197.0,-109.0,225.0,-7.0,23.0,-0.30,0.96,-1.0,7.0,-5.0,23.0,-2.0,2.0,0.0,0.0,-2.0,6.0,4.0,10.0,1.0,1.0,5.0,21.0,-1.0,1.0,-15.0,31.0,-8.0,20.0,-7.0,11.0,-0.64,1.00,-4.0,4.0,-2.0,2.0,-26.0,100.0,1.044048,2,3.103652,20.120048,0.005754,20.000000,56.000000,1.024390,2
3,0.37,0.49,4.0,8.0,1.0,1.0,1.0,1.0,56.0,380.0,0.04,1.64,-1.0,9.0,-1.0,3.0,-1.0,3.0,3.0,3.0,2.0,4.0,-1.0,1.0,1.0,1.0,2.0,6.0,2.0,2.0,56.0,462.0,-14.0,202.0,70.0,178.0,-4.0,32.0,-0.01,0.97,4.0,4.0,-2.0,18.0,-2.0,2.0,0.0,0.0,-3.0,5.0,-4.0,12.0,1.0,1.0,-6.0,20.0,-3.0,3.0,-5.0,39.0,1.0,23.0,-6.0,16.0,-0.36,1.02,3.0,7.0,-1.0,1.0,12.0,100.0,1.064560,2,0.093617,0.235317,0.018924,3.500000,6.500000,1.600000,1
5,-0.95,1.57,1.0,9.0,-2.0,2.0,0.0,0.0,-21.0,353.0,-0.02,1.64,0.0,10.0,1.0,1.0,2.0,4.0,2.0,2.0,-1.0,5.0,0.0,2.0,0.0,0.0,0.0,8.0,1.0,1.0,-19.0,431.0,-59.0,183.0,38.0,170.0,-1.0,29.0,0.20,1.14,0.0,4.0,5.0,19.0,0.0,2.0,0.0,0.0,-4.0,10.0,4.0,16.0,0.0,2.0,-4.0,8.0,0.0,2.0,-5.0,37.0,-6.0,26.0,1.0,11.0,0.05,1.05,0.0,6.0,-1.0,3.0,-4.0,100.0,1.092484,2,0.131783,0.154580,0.049411,3.126923,4.500000,1.816667,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,-0.31,0.69,1.0,9.0,-2.0,2.0,-1.0,1.0,-45.0,385.0,-0.01,1.63,-1.0,11.0,1.0,1.0,-3.0,7.0,0.0,4.0,-1.0,3.0,2.0,2.0,0.0,0.0,0.0,6.0,1.0,3.0,-54.0,472.0,-31.0,253.0,-14.0,132.0,3.0,31.0,0.1

In [438]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df.iloc[:, -1], test_size = 0.30)

In [405]:
selector = SelectKBest(f_classif, k=30)
selector.fit(X_train, y_train)
cols = selector.get_support(indices=True)
X_train_new = X_train.iloc[:,cols]
X_test_new = X_test.iloc[:,cols]

C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:115: UserWarning: Features [71] are constant.
  UserWarning)
C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [439]:
X_train_n = tf.keras.utils.normalize(X_train, axis = 1)
X_test_n = tf.keras.utils.normalize(X_test, axis = 1)

In [449]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(6, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(3, activation = tf.nn.softmax))
es = EarlyStopping(monitor='accuracy', mode='min', verbose=1)
model.compile(optimizer = 'adam',
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])
model.fit(X_train_n, y_train, epochs = 150)

Epoch 1/150
6/6 [==============================] - 0s 2ms/step - loss: 1.0977 - accuracy: 0.4010
Epoch 2/150
6/6 [==============================] - 0s 3ms/step - loss: 1.0944 - accuracy: 0.4375
Epoch 3/150
6/6 [==============================] - 0s 2ms/step - loss: 1.0918 - accuracy: 0.4375
Epoch 4/150
6/6 [==============================] - 0s 2ms/step - loss: 1.0896 - accuracy: 0.4375
Epoch 5/150
6/6 [==============================] - 0s 2ms/step - loss: 1.0882 - accuracy: 0.4375
Epoch 6/150
6/6 [==============================] - 0s 2ms/step - loss: 1.0862 - accuracy: 0.4375
Epoch 7/150
6/6 [==============================] - 0s 2ms/step - loss: 1.0847 - accuracy: 0.4375
Epoch 8/150
6/6 [==============================] - 0s 3ms/step - loss: 1.0831 - accuracy: 0.4375
Epoch 9/150
6/6 [==============================] - 0s 2ms/step - loss: 1.0814 - accuracy: 0.4375
Epoch 10/150
6/6 [==============================] - 0s 2ms/step - loss: 1.0798 - accuracy: 0.4375
Epoch 11/150
6/6 [===========

6/6 [==============================] - 0s 2ms/step - loss: 0.9784 - accuracy: 0.5156
Epoch 85/150
6/6 [==============================] - 0s 3ms/step - loss: 0.9774 - accuracy: 0.5156
Epoch 86/150
6/6 [==============================] - 0s 3ms/step - loss: 0.9762 - accuracy: 0.5208
Epoch 87/150
6/6 [==============================] - 0s 2ms/step - loss: 0.9749 - accuracy: 0.5156
Epoch 88/150
6/6 [==============================] - 0s 2ms/step - loss: 0.9739 - accuracy: 0.5156
Epoch 89/150
6/6 [==============================] - 0s 2ms/step - loss: 0.9731 - accuracy: 0.5208
Epoch 90/150
6/6 [==============================] - 0s 2ms/step - loss: 0.9715 - accuracy: 0.5208
Epoch 91/150
6/6 [==============================] - 0s 4ms/step - loss: 0.9707 - accuracy: 0.5208
Epoch 92/150
6/6 [==============================] - 0s 8ms/step - loss: 0.9694 - accuracy: 0.5208
Epoch 93/150
6/6 [==============================] - 0s 4ms/step - loss: 0.9683 - accuracy: 0.5312
Epoch 94/150
6/6 [===============

In [444]:
model.evaluate(X_test_n, y_test)

3/3 [==============================] - 0s 2ms/step - loss: 0.8847 - accuracy: 0.6024


[0.8847261071205139, 0.6024096608161926]

In [448]:
preds = pd.DataFrame(model.predict(X_test_n), columns = ['pred 0','pred 1','pred 2'])
predicted_df = pd.concat([X_test.reset_index(drop = True),preds], axis = 1).iloc[:,-6:]
bo_po_dha = list(zip(predicted_df.columns[0:3],predicted_df.columns[-3:],['draw','home','away']))
for book_odds, pred_odds, dha in bo_po_dha:
    predicted_df[dha] = predicted_df.apply(lambda x: kelly(x[pred_odds], x[book_odds], 1), axis=1)
predicted_df['target'] = y_test.reset_index(drop=True)
dha = {'draw':'0','home':'1','away':'2'}
predicted_df = predicted_df.replace('no value', 0)
predicted_df[['draw','home','away']] = predicted_df[['draw','home','away']].astype(float)
for word, number in dha.items():
    predicted_df[f'{word}_profits'] = np.where(predicted_df['target'] == int(number),
                                    predicted_df[word] * (predicted_df['med_'+number] - 1) * 100,
                                    -100*predicted_df[word])

predicted_df['profits'] = predicted_df[predicted_df.columns[-3:]].sum(axis = 1)
p = round(np.sum(predicted_df["profits"]),3)
predicted_df = predicted_df.round(3)

print(f'number of bets made: {predicted_df.shape[0]}\n')
print(f'profit on $100 bankroll: ${p}\n')
print(predicted_df.sum()[-4:-1])

3/3 [==============================] - 0s 2ms/step
number of bets made: 83

profit on $100 bankroll: $-985.582

draw_profits   -168.024
home_profits   -318.615
away_profits   -498.943
dtype: float64


In [447]:
pd.concat([X_test.reset_index(drop = True).iloc[:,:-3], predicted_df], axis = 1).sort_values('home_profits')

,expected_goals_(xg)_diff,expected_goals_(xg)_sum,total_shots_diff,total_shots_sum,big_chances_diff,big_chances_sum,big_chances_missed_diff,big_chances_missed_sum,accurate_passes_diff,accurate_passes_sum,accurate_passes_percentage_diff,accurate_passes_percentage_sum,fouls_committed_diff,fouls_committed_sum,offsides_diff,offsides_sum,corners_diff,corners_sum,shots_off_target_diff,shots_off_target_sum,shots_on_target_diff,shots_on_target_sum,blocked_shots_diff,blocked_shots_sum,hit_woodwork_diff,hit_woodwork_sum,shots_inside_box_diff,shots_inside_box_sum,shots_outside_box_diff,shots_outside_box_sum,passes_diff,passes_sum,own_half_diff,own_half_sum,opposition_half_diff,opposition_half_sum,accurate_long_balls_diff,accurate_long_balls_sum,accurate_long_balls_percentage_diff,accurate_long_balls_percentage_sum,accurate_crosses_diff,accurate_crosses_sum,throws_diff,throws_sum,yellow_cards_diff,yellow_cards_sum,red_cards_diff,red_cards_sum,tackles_won_diff,tackles_won_sum,interceptions_diff,interceptions_sum,blocks_diff,blocks_sum,clearances_diff,clearances_sum,keeper_saves_diff,keeper_saves_sum,duels_won_diff,duels_won_sum,ground_duels_won_diff,ground_duels_won_sum,aerial_duels_won_diff,aerial_duels_won_sum,aerial_duels_won_percentage_diff,aerial_duels_won_percentage_sum,successful_dribbles_diff,successful_dribbles_sum,score_diff,score_sum,posession_diff,posession_sum,odds_sum,odds_predict,std_0,std_1,std_2,med_0,med_1,med_2,pred 0,pred 1,pred 2,draw,home,away,target,draw_profits,home_profits,away_profits,profits
56,1.47,1.71,11.0,19.0,3.0,3.0,3.0,3.0,165.0,303.0,0.24,1.42,-1.0,11.0,0.0,4.0,-1.0,3.0,5.0,7.0,3.0,7.0,3.0,5.0,1.0,1.0,6.0,12.0,5.0,7.0,166.0,400.0,80.0,164.0,85.0,139.0,11.0,35.0,0.33,0.91,1.0,3.0,1.0,15.0,-1.0,1.0,0.0,0.0,2.0,8.0,-4.0,4.0,-4.0,6.0,-8.0,18.0,-2.0,6.0,6.0,52.0,2.0,26.0,4.0,26.0,0.24,1.12,3.0,5.0,0.0,0.0,42.0,100.0,1.078057,1,0.025713,0.022268,0.141794,2.375,2.300,4.500,0.158,0.740,0.102,0.0,0.540,0.000,0,0.0,-54.020,-0.00,-54.020
57,-1.46,1.74,1.0,7.0,-2.0,2.0,0.0,0.0,146.0,394.0,0.10,1.74,-3.0,21.0,1.0,1.0,-1.0,5.0,2.0,2.0,-1.0,3.0,0.0,2.0,0.0,0.0,1.0,5.0,0.0,2.0,143.0,447.0,31.0,199.0,115.0,195.0,-1.0,33.0,0.10,1.36,-2.0,2.0,-4.0,12.0,-1.0,1.0,0.0,0.0,1.0,9.0,-5.0,7.0,0.0,2.0,-1.0,13.0,0.0,0.0,-3.0,47.0,1.0,35.0,-4.0,12.0,-0.34,1.00,-1.0,3.0,-1.0,3.0,32.0,100.0,1.074359,2,0.070662,0.292731,0.031995,3.250,6.000,1.667,0.224,0.543,0.234,0.0,0.451,0.000,2,-0.0,-45.110,0.00,-45.110
13,0.89,1.11,7.0,9.0,1.0,1.0,0.0,0.0,77.0,309.0,0.06,1.68,-3.0,13.0,1.0,1.0,1.0,5.0,4.0,4.0,0.0,2.0,3.0,3.0,0.0,0.0,7.0,9.0,0.0,0.0,77.0,365.0,43.0,183.0,34.0,126.0,8.0,24.0,0.13,0.97,3.0,5.0,2.0,14.0,-2.0,2.0,0.0,0.0,-3.0,13.0,2.0,8.0,-3.0,3.0,-1.0,19.0,0.0,0.0,-2.0,54.0,-1.0,43.0,-1.0,11.0,-0.10,1.00,4.0,10.0,0.0,2.0,20.0,100.0,1.091726,0,0.075452,0.045693,0.212633,2.300,2.388,4.200,0.198,0.667,0.135,0.0,0.428,0.000,0,0.0,-42.780,-0.00,-42.780
5,0.41,2.49,5.0,13.0,1.0,3.0,1.0,1.0,19.0,353.0,0.04,1.76,0.0,16.0,0.0,0.0,3.0,5.0,0.0,4.0,3.0,5.0,2.0,4.0,0.0,0.0,3.0,9.0,2.0,4.0,12.0,400.0,-74.0,188.0,93.0,165.0,16.0,32.0,0.45,1.33,6.0,8.0,0.0,14.0,0.0,2.0,0.0,0.0,2.0,6.0,-1.0,11.0,-2.0,4.0,-4.0,10.0,-3.0,3.0,3.0,39.0,1.0,27.0,2.0,12.0,0.16,1.00,0.0,2.0,0.0,2.0,6.0,100.0,1.069863,0,0.061407,0.093391,0.084993,2.300,3.100,3.200,0.260,0.602,0.138,0.0,0.412,0.000,0,0.0,-41.180,-0.00,-41.180
14,0.18,0.82,3.0,9.0,0.0,2.0,0.0,2.0,63.0,345.0,0.04,1.54,-3.0,5.0,-1.0,1.0,-2.0,6.0,1.0,5.0,1.0,3.0,1.0,1.0,0.0,0.0,1.0,5.0,2.0,4.0,70.0,444.0,49.0,169.0,14.0,176.0,-5.0,25.0,-0.20,0.92,-2.0,4.0,-5.0,31.0,0.0,0.0,0.0,0.0,-3.0,11.0,2.0,8.0,-1.0,1.0,2.0,18.0,-1.0,3.0,4.0,44.0,1.0,27.0,3.0,17.0,0.18,1.00,0.0,4.0,0.0,0.0,12.0,100.0,1.060258,2,0.075028,0.283061,0.052489,2.600,4.750,2.150,0.221,0.482,0.297,0.0,0.344,0.000,2,-0.0,-34.390,0.00,-34.390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...